In [1]:
# import standard Python libraries
import sqlite3
import pandas as pd
import numpy as np
from graphlib import TopologicalSorter # for rank-ordering courses

In [3]:
## import catalog functions
import utils
from utils import TimedSQLiteConnection, get_query, get_query_one, get_query_dict, get_query_course_dict, get_query_df

In [4]:
## create sqlite connection
conn = TimedSQLiteConnection('UMGC.db')
timedConnection = conn # so I can test function code directly

#### Fix the student_info dictionary to follow current catalog practice

In [5]:
student_info = utils.initialize_student_info()
student_info['ge'] = utils.initialize_ge() # only if undergraduate
student_info_populated = False # may be needed later 
student_data = utils.initialize_student_data() # will have required, periods, schedule

student_info['user_id'] = 0
student_info

{'user_id': 0,
 'name': None,
 'financial_aid': None,
 'resident_status': None,
 'student_profile': None,
 'transfer_credits': None,
 'app_stage': None,
 'app_stage_id': None,
 'first_term': None,
 'program_id': None,
 'degree_program': None,
 'menu': {'degree': None, 'area_of_study': None, 'program': None},
 'ge': {'arts': {'1': None, '2': None, 'nopre': False},
  'beh': {'1': None, '2': None, 'nopre': False},
  'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
  'comm': {'1': 'WRTG 111',
   '2': 'WRTG 112',
   '3': None,
   '4': None,
   'nopre': False},
  'math': {'1': None, 'nopre': False},
  'res': {'1': None,
   '2': 'LIBS 150',
   '3': None,
   '3a': None,
   '3b': None,
   '3c': None,
   'nopre': False}}}

In [6]:
# Step 1. Student name registered

# from utils.set_user_vars_given_role when retrieving from database
# here it is entered and registered
student_info['name'] = 'Joe Smith'

student_info['app_stage_id'] = 1
student_info['app_stage'] = 'new' # get this from query

student_info

{'user_id': 0,
 'name': 'Joe Smith',
 'financial_aid': None,
 'resident_status': None,
 'student_profile': None,
 'transfer_credits': None,
 'app_stage': 'new',
 'app_stage_id': 1,
 'first_term': None,
 'program_id': None,
 'degree_program': None,
 'menu': {'degree': None, 'area_of_study': None, 'program': None},
 'ge': {'arts': {'1': None, '2': None, 'nopre': False},
  'beh': {'1': None, '2': None, 'nopre': False},
  'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
  'comm': {'1': 'WRTG 111',
   '2': 'WRTG 112',
   '3': None,
   '4': None,
   'nopre': False},
  'math': {'1': None, 'nopre': False},
  'res': {'1': None,
   '2': 'LIBS 150',
   '3': None,
   '3a': None,
   '3b': None,
   '3c': None,
   'nopre': False}}}

In [7]:
# Step 2. Student information

student_info['financial_aid'] = 1
student_info['transfer_credits'] = 0
student_info['student_profile'] = 'Full-time' # check spelling
student_info['resident_status'] = 'In-State' # check spelling
student_info['first_term'] = 'Spring 2024'

student_info['app_stage_id'] = 2
student_info['app_stage'] = 'personalized' # get this from query

student_info

{'user_id': 0,
 'name': 'Joe Smith',
 'financial_aid': 1,
 'resident_status': 'In-State',
 'student_profile': 'Full-time',
 'transfer_credits': 0,
 'app_stage': 'personalized',
 'app_stage_id': 2,
 'first_term': 'Spring 2024',
 'program_id': None,
 'degree_program': None,
 'menu': {'degree': None, 'area_of_study': None, 'program': None},
 'ge': {'arts': {'1': None, '2': None, 'nopre': False},
  'beh': {'1': None, '2': None, 'nopre': False},
  'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
  'comm': {'1': 'WRTG 111',
   '2': 'WRTG 112',
   '3': None,
   '4': None,
   'nopre': False},
  'math': {'1': None, 'nopre': False},
  'res': {'1': None,
   '2': 'LIBS 150',
   '3': None,
   '3a': None,
   '3b': None,
   '3c': None,
   'nopre': False}}}

In [8]:
# Step 3. Choose program

student_info['menu']['degree'] = 2
student_info['menu']['area_of_study'] = 1
student_info['menu']['program'] = 2
student_info['program_id'] = 2
student_info['degree_program'] = 'BS in Accounting'

student_info['app_stage_id'] = 3
student_info['app_stage'] = 'program chosen' # get this from query

student_info

{'user_id': 0,
 'name': 'Joe Smith',
 'financial_aid': 1,
 'resident_status': 'In-State',
 'student_profile': 'Full-time',
 'transfer_credits': 0,
 'app_stage': 'program chosen',
 'app_stage_id': 3,
 'first_term': 'Spring 2024',
 'program_id': 2,
 'degree_program': 'BS in Accounting',
 'menu': {'degree': 2, 'area_of_study': 1, 'program': 2},
 'ge': {'arts': {'1': None, '2': None, 'nopre': False},
  'beh': {'1': None, '2': None, 'nopre': False},
  'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
  'comm': {'1': 'WRTG 111',
   '2': 'WRTG 112',
   '3': None,
   '4': None,
   'nopre': False},
  'math': {'1': None, 'nopre': False},
  'res': {'1': None,
   '2': 'LIBS 150',
   '3': None,
   '3a': None,
   '3b': None,
   '3c': None,
   'nopre': False}}}

In [9]:
student_info['schedule']= {}
schedule = student_info['schedule']

schedule['courses_per_session'] = 3
schedule['max_credits'] = 18
schedule['attend_summer'] = True
schedule['sessions'] = ['Session 1', 'Session 3']

schedule

{'courses_per_session': 3,
 'max_credits': 18,
 'attend_summer': True,
 'sessions': ['Session 1', 'Session 3']}

### Create periods object to store slots we will put courses into

In [10]:
periods_df = utils.generate_periods(
    summer=False,
)
#    #start_term='student_info['first_term']',
#    start_term='SPRING 2024',
#    max_courses=schedule['courses_per_session'],
#    max_credits=schedule['max_credits'],
#    summer=schedule['attend_summer'],
#    sessions=schedule['sessions'],
#    as_df=True
#)
periods_df
periods_df['period'] = periods_df['term'] + ' ' + periods_df['year'].astype(str)

periods_df

,id,term,session,year,max_courses,max_credits,previous,period
0,1,SPRING,1,2024,3,18,1,SPRING 2024
1,2,SPRING,2,2024,0,0,2,SPRING 2024
2,3,SPRING,3,2024,3,18,2,SPRING 2024
3,4,SUMMER,1,2024,0,0,1,SUMMER 2024
4,5,SUMMER,2,2024,0,0,2,SUMMER 2024
...,...,...,...,...,...,...,...,...
80,81,SUMMER,1,2031,0,0,1,SUMMER 2031
81,82,SUMMER,2,2031,0,0,2,SUMMER 2031
82,83,FALL,1,2031,3,18,1,FALL 2031
83,84,FALL,2,2031,0,0,2,FALL 2031


In [11]:
# Keep track of max credits per (term, year), e.g., 'SPRING 2024',
# since credit constraints are officially managed at that level rather
# than as the (term, year, session) level

max_credits_df = periods_df.groupby(['year', 'term', 'period'])['max_credits'].max().reset_index()
#max_credits_df = periods_df.groupby(['period'])['max_credits'].max().reset_index()

del periods_df['max_credits']

periods_df

,id,term,session,year,max_courses,previous,period
0,1,SPRING,1,2024,3,1,SPRING 2024
1,2,SPRING,2,2024,0,2,SPRING 2024
2,3,SPRING,3,2024,3,2,SPRING 2024
3,4,SUMMER,1,2024,0,1,SUMMER 2024
4,5,SUMMER,2,2024,0,2,SUMMER 2024
...,...,...,...,...,...,...,...
80,81,SUMMER,1,2031,0,1,SUMMER 2031
81,82,SUMMER,2,2031,0,2,SUMMER 2031
82,83,FALL,1,2031,3,1,FALL 2031
83,84,FALL,2,2031,0,2,FALL 2031


In [12]:
print(f'periods_df: \n{periods_df.head()}')

print(f'\nmax_credits_df:\n{max_credits_df.head()}')

periods_df: 
   id    term  session  year  max_courses  previous       period
0   1  SPRING        1  2024            3         1  SPRING 2024
1   2  SPRING        2  2024            0         2  SPRING 2024
2   3  SPRING        3  2024            3         2  SPRING 2024
3   4  SUMMER        1  2024            0         1  SUMMER 2024
4   5  SUMMER        2  2024            0         2  SUMMER 2024

max_credits_df:
   year    term       period  max_credits
0  2024    FALL    FALL 2024           18
1  2024  SPRING  SPRING 2024           18
2  2024  SUMMER  SUMMER 2024            0
3  2025    FALL    FALL 2025           18
4  2025  SPRING  SPRING 2025           18


In [ ]:
# Note that handle_prerequisites is called within generate_schedule, 
# so there is no need to handle it ourselves.
#
# `course_df = await handle_prerequisites(timedConnection, course_df)`
#
# `schedule = await generate_schedule(timedConnection, course_df, periods_df)`
#
# Updating `periods_df` will have the largest impact on scheduling.

In [40]:
async def rename_elective(df):
    # If a value is ELECTIVE, replace with ELECTIVE 1, ELECTIVE 2, etc. so that names are unique
    # Identify rows where 'name' is 'ELECTIVE'
    elective_rows = df['name'] == 'ELECTIVE'
    
    # Create a series of unique numbers starting from 1
    elective_numbers = range(1, elective_rows.sum() + 1)
    
    # Assign these numbers to the 'ELECTIVE' rows
    df.loc[elective_rows, 'name'] = ['ELECTIVE ' + str(num) for num in elective_numbers]
    return df

In [41]:
async def get_df_for_figure(timedConnection, program_id=10):

    query = 'SELECT * FROM catalog_program_sequence_view WHERE program_id = ?'
    #program_id = 10
    df = await get_query_df(timedConnection, query, params=(program_id,))

    # this is not really necessary
    # make sure 'credits' is numeric rather than string
    df['credits'] = pd.to_numeric(df['credits'])
    # change column name from 'course' to 'name'
    df.rename(columns={'course': 'name'}, inplace=True)
    # replace possible None values
    columns_to_fill = ['title', 'pre', 'pre_credits', 'substitutions', 'description']
    df[columns_to_fill] = df[columns_to_fill].fillna('')

    # add a scheduled boolean to keep track of which courses have been scheduled
    df['scheduled'] = False
    # change 'locked' to boolean (from 0/1 from sql query)
    df['locked'] = df['locked'].astype(bool)

    # rename electives
    df = await rename_elective(df)
    
    return df

In [42]:
df = await get_df_for_figure(timedConnection, 10)
df

,program_id,seq,name,course_type,type,credits,title,completed,term,year,session,locked,pre,pre_credits,substitutions,description,scheduled
0,10,1,LIBS 150,general,general,1,Introduction to Research,0,0,0,0,False,,,"COMP 111, LIBS 100, or LIBS 150",An introduction to the research process and me...,False
1,10,2,PACE 111T,general,general,3,Program and Career Exploration in Technology,0,0,0,0,False,,,"PACE 111B, PACE 111C, PACE 111M, PACE 111P, PA...",(Fulfills the general education requirement in...,False
2,10,3,WRTG 111,general,general,3,Academic Writing I,0,0,0,0,False,,,"WRTG 100A, WRTG 111, or WRTG 111X",(The first course in the two-course series WRT...,False
3,10,4,IFSM 201,general,general,3,Concepts and Applications of Information Techn...,0,0,0,0,False,,,"BMGT 301, CAPP 101, CAPP 300, CMST 300, IFSM 2...",(Access to a standard office productivity pack...,False
4,10,5,NUTR 100,general,general,3,Elements of Nutrition,0,0,0,0,False,,,NUTR 100 or NUTR 200,A study of the scientific and quantitative fou...,False
5,10,6,CMIT 202,major,major,3,Fundamentals of Computer Troubleshooting,0,0,0,0,False,,,,(Designed to help prepare for the CompTIA A+ e...,False
6,10,7,SPCH 100,general,general,3,Foundations of Oral Communication,0,0,0,0,False,,,"SPCH 100, SPCH 100X, SPCH 101, SPCH 107, or SP...","For online sections, access to a broadband int...",False
7,10,8,MATH 107,general,general,3,College Algebra,0,0,0,0,False,,,MATH 107 or MATH 115,(The first course in the two-course series MAT...,False
8,10,9,WRTG 112,general,general,3,Academic Writing II,0,0,0,0,False,,,"ENGL 101, ENGL 101X, WRTG 101, WRTG 101S, WRTG...",(The second course in the two-course series WR...,False
9,10,10,CMIT 265,major,major,3,Fundamentals of Networking,0,0,0,0,False,(CMIT 202 | CMSC 115 | CMIS 141),,CMIT 265 or CMIT 265M,(Designed to help prepare for the CompTIA Netw...,False


In [43]:
course_df = df.copy()
#course_df = await utils.handle_prerequisites(timedConnection, course_df)
# Note: handle_prerequisites is removing electives, etc. instead. I need to fix it in the future.

new_df = course_df[['name', 'pre']].copy()
courses = new_df.to_dict(orient='records')

# Convert list of courses to a dictionary for easy lookup
courses_dict = {course['name']: course['pre'] for course in courses}
courses_dict

{'LIBS 150': '',
 'PACE 111T': '',
 'WRTG 111': '',
 'IFSM 201': '',
 'NUTR 100': '',
 'CMIT 202': '',
 'SPCH 100': '',
 'MATH 107': '',
 'WRTG 112': '',
 'CMIT 265': '(CMIT 202 | CMSC 115 | CMIS 141)',
 'HIST 125': '',
 'BIOL 103': '',
 'BEHS 103': '',
 'ARTH 334': '',
 'ELECTIVE 1': '',
 'ECON 103': '',
 'CMIT 291': '(CMIT 202 | CMIT 265)',
 'CMIT 320': 'CMIT 265',
 'ELECTIVE 2': '',
 'CMIT 321': 'CMIT 320',
 'ELECTIVE 3': '',
 'CMIT 351': 'CMIT 265',
 'ELECTIVE 4': '',
 'CMIT 326': '',
 'WRTG 393': 'WRTG 112*',
 'ELECTIVE 5': '',
 'ELECTIVE 6': '',
 'ELECTIVE 7': '',
 'CMIT 421': 'CMIT 320',
 'ELECTIVE 8': '',
 'ELECTIVE 9': '',
 'CMIT 386': 'CMIT 321',
 'ELECTIVE 10': '',
 'ELECTIVE 11': '',
 'CCJS 321': '',
 'ELECTIVE 12': '',
 'ELECTIVE 13': '',
 'ELECTIVE 14': '',
 'ELECTIVE 15': '',
 'CMIT 495': '',
 'CAPL 398A': ''}

In [44]:
courses

[{'name': 'LIBS 150', 'pre': ''},
 {'name': 'PACE 111T', 'pre': ''},
 {'name': 'WRTG 111', 'pre': ''},
 {'name': 'IFSM 201', 'pre': ''},
 {'name': 'NUTR 100', 'pre': ''},
 {'name': 'CMIT 202', 'pre': ''},
 {'name': 'SPCH 100', 'pre': ''},
 {'name': 'MATH 107', 'pre': ''},
 {'name': 'WRTG 112', 'pre': ''},
 {'name': 'CMIT 265', 'pre': '(CMIT 202 | CMSC 115 | CMIS 141)'},
 {'name': 'HIST 125', 'pre': ''},
 {'name': 'BIOL 103', 'pre': ''},
 {'name': 'BEHS 103', 'pre': ''},
 {'name': 'ARTH 334', 'pre': ''},
 {'name': 'ELECTIVE 1', 'pre': ''},
 {'name': 'ECON 103', 'pre': ''},
 {'name': 'CMIT 291', 'pre': '(CMIT 202 | CMIT 265)'},
 {'name': 'CMIT 320', 'pre': 'CMIT 265'},
 {'name': 'ELECTIVE 2', 'pre': ''},
 {'name': 'CMIT 321', 'pre': 'CMIT 320'},
 {'name': 'ELECTIVE 3', 'pre': ''},
 {'name': 'CMIT 351', 'pre': 'CMIT 265'},
 {'name': 'ELECTIVE 4', 'pre': ''},
 {'name': 'CMIT 326', 'pre': ''},
 {'name': 'WRTG 393', 'pre': 'WRTG 112*'},
 {'name': 'ELECTIVE 5', 'pre': ''},
 {'name': 'ELECTIVE

In [ ]:
async def prep_df_for_new_order(df):

    new_df = df[['name','pre']].copy()
    ## process prerequisites
    ## right now might not work for &'s, will need to rewrite

    data = new_df.to_dict(orient='records')
    # Convert to the desired format
    courses = {}
    for item in data:
        name = item['name']
        pre = item['pre']
        # Check if the course name ends with a star
        if name.endswith('*'):
            # If it does, remove the star
            name = name[:-1]
        if pre.endswith('*'):
            # If it does, remove the star
            pre = pre[:-1]
        if pre == '':
            courses[name] = []
        else:
            # Remove parentheses and split by '|'
            prerequisites = pre.strip('()').split(' | ')
            # Each prerequisite is a list of one item
            prerequisites = [[p.strip()] for p in prerequisites]
            courses[name] = prerequisites

    return courses

In [ ]:
def parse_prerequisites(pre):
    # convert to a format for topological sorting
    def split_conditions(cond):
        if '&' in cond:
            return [c.strip() for c in cond.split('&')]
        return [cond.strip()]
    
    if '(' not in pre:
        return [[c.strip()] for c in pre.split('|')]
    
    stack = []
    current = []
    for char in pre:
        if char == '(':
            stack.append(current)
            current = []
        elif char == ')':
            condition = current
            current = stack.pop()
            if current and current[-1] == '&':
                current.pop()
                current[-1].extend(split_conditions(''.join(condition)))
            else:
                current.append(split_conditions(''.join(condition)))
        elif char in ('|', '&'):
            if current and current[-1] not in ('|', '&'):
                current.append(''.join(current))
            current.append(char)
        else:
            current.append(char)
    
    return [[c] if isinstance(c, str) else c for c in current if c not in ('|', '&')]

In [ ]:
# Convert to the desired format
courses = {}
for item in data:
    name = item['name']
    pre = item['pre']
    # Check if the course name ends with a star
    if name.endswith('*'):
        # If it does, remove the star
        name = name[:-1]
    if pre.endswith('*'):
        # If it does, remove the star
        pre = pre[:-1]
    if pre == '':
        courses[name] = []
    else:
        courses[name] = parse_prerequisites(pre)

import pprint
pprint.pprint(courses)

In [45]:
await prep_df_for_new_order(df)

{'LIBS 150': [],
 'PACE 111T': [],
 'WRTG 111': [],
 'IFSM 201': [],
 'NUTR 100': [],
 'CMIT 202': [],
 'SPCH 100': [],
 'MATH 107': [],
 'WRTG 112': [],
 'CMIT 265': [['CMIT 202'], ['CMSC 115'], ['CMIS 141']],
 'HIST 125': [],
 'BIOL 103': [],
 'BEHS 103': [],
 'ARTH 334': [],
 'ELECTIVE 1': [],
 'ECON 103': [],
 'CMIT 291': [['CMIT 202'], ['CMIT 265']],
 'CMIT 320': [['CMIT 265']],
 'ELECTIVE 2': [],
 'CMIT 321': [['CMIT 320']],
 'ELECTIVE 3': [],
 'CMIT 351': [['CMIT 265']],
 'ELECTIVE 4': [],
 'CMIT 326': [],
 'WRTG 393': [['WRTG 112']],
 'ELECTIVE 5': [],
 'ELECTIVE 6': [],
 'ELECTIVE 7': [],
 'CMIT 421': [['CMIT 320']],
 'ELECTIVE 8': [],
 'ELECTIVE 9': [],
 'CMIT 386': [['CMIT 321']],
 'ELECTIVE 10': [],
 'ELECTIVE 11': [],
 'CCJS 321': [],
 'ELECTIVE 12': [],
 'ELECTIVE 13': [],
 'ELECTIVE 14': [],
 'ELECTIVE 15': [],
 'CMIT 495': [],
 'CAPL 398A': []}

In [ ]:
async def return_order_old(courses):
    from graphlib import TopologicalSorter

    classes_flat = {course: [item for sublist in prereqs for item in sublist] for course, prereqs in courses.items()}
    ts = TopologicalSorter(classes_flat)
    order = list(ts.static_order())
    return order

In [30]:
async def return_order(courses):
    # Step 1: Include courses that are prerequisites and present in the courses dictionary
    courses_with_prereqs = {}
    for course, prereqs in courses.items():
        # Flatten the list of prerequisites and include only those present in the courses dictionary
        flattened_prereqs = [prereq for sublist in prereqs for prereq in sublist if prereq in courses]
        if flattened_prereqs or course in courses:
            courses_with_prereqs[course] = flattened_prereqs

    # Step 2: Identify courses without prerequisites
    courses_without_prereqs = [course for course, prereqs in courses.items() if not prereqs and course not in {c for sublist in courses.values() for prereq_list in sublist for c in prereq_list}]

    # Step 3: Perform topological sort on courses with prerequisites
    ts = TopologicalSorter(courses_with_prereqs)
    sorted_courses_with_prereqs = list(ts.static_order())

    # Step 4: Combine sorted courses with the original list of courses without prerequisites
    # Create a dataframe to preserve the original order
    df = pd.DataFrame(list(courses.keys()), columns=['course'])
    df['sorted_order'] = -1  # Initialize with -1

    # Assign sorted order to courses with prerequisites
    for i, course in enumerate(sorted_courses_with_prereqs):
        df.loc[df['course'] == course, 'sorted_order'] = i

    # Assign original positions to courses without prerequisites
    for course in courses_without_prereqs:
        df.loc[df['course'] == course, 'sorted_order'] = df.index[df['course'] == course].tolist()[0] + len(sorted_courses_with_prereqs)

    # Sort dataframe by the 'sorted_order' column
    df.sort_values(by='sorted_order', inplace=True)

    # Return the sorted order of courses
    return df['course'].tolist()


In [27]:
# Example usage
courses = {
    'LIBS 150': [],
    'PACE 111T': [],
    'WRTG 111': [],
    'IFSM 201': [],
    'NUTR 100': [],
    'CMIT 202': [],
    'SPCH 100': [],
    'MATH 107': [],
    'WRTG 112': [],
    'CMIT 265': [['CMIT 202'], ['CMSC 115'], ['CMIS 141']],
    'HIST 125': [],
    'BIOL 103': [],
    'BEHS 103': [],
    'ARTH 334': [],
    'ELECTIVE 1': [],
    'ECON 103': [],
    'CMIT 291': [['CMIT 202'], ['CMIT 265']],
    'CMIT 320': [['CMIT 265']],
    'ELECTIVE 2': [],
    'CMIT 321': [['CMIT 320']],
    'ELECTIVE 3': [],
    'CMIT 351': [['CMIT 265']],
    'ELECTIVE 4': [],
    'CMIT 326': [],
    'WRTG 393': [['WRTG 112']],
    'ELECTIVE 5': [],
    'ELECTIVE 6': [],
    'ELECTIVE 7': [],
    'CMIT 421': [['CMIT 320']],
    'ELECTIVE 8': [],
    'ELECTIVE 9': [],
    'CMIT 386': [['CMIT 321']],
    'ELECTIVE 10': [],
    'ELECTIVE 11': [],
    'CCJS 321': [],
    'ELECTIVE 12': [],
    'ELECTIVE 13': [],
    'ELECTIVE 14': [],
    'ELECTIVE 15': [],
    'CMIT 495': [],
    'CAPL 398A': []
}

In [32]:
courses_with_prereqs = {}

In [33]:
for course, prereqs in courses.items():
    print("Course: {}, prereqs: {}".format(course, prereqs))

Course: LIBS 150, prereqs: []
Course: PACE 111T, prereqs: []
Course: WRTG 111, prereqs: []
Course: IFSM 201, prereqs: []
Course: NUTR 100, prereqs: []
Course: CMIT 202, prereqs: []
Course: SPCH 100, prereqs: []
Course: MATH 107, prereqs: []
Course: WRTG 112, prereqs: []
Course: CMIT 265, prereqs: [['CMIT 202'], ['CMSC 115'], ['CMIS 141']]
Course: HIST 125, prereqs: []
Course: BIOL 103, prereqs: []
Course: BEHS 103, prereqs: []
Course: ARTH 334, prereqs: []
Course: ELECTIVE 1, prereqs: []
Course: ECON 103, prereqs: []
Course: CMIT 291, prereqs: [['CMIT 202'], ['CMIT 265']]
Course: CMIT 320, prereqs: [['CMIT 265']]
Course: ELECTIVE 2, prereqs: []
Course: CMIT 321, prereqs: [['CMIT 320']]
Course: ELECTIVE 3, prereqs: []
Course: CMIT 351, prereqs: [['CMIT 265']]
Course: ELECTIVE 4, prereqs: []
Course: CMIT 326, prereqs: []
Course: WRTG 393, prereqs: [['WRTG 112']]
Course: ELECTIVE 5, prereqs: []
Course: ELECTIVE 6, prereqs: []
Course: ELECTIVE 7, prereqs: []
Course: CMIT 421, prereqs: [['C

In [ ]:
for course, prereqs in courses.items():
    # Flatten the list of prerequisites and include only those present in the courses dictionary
    flattened_prereqs = [prereq for sublist in prereqs for prereq in sublist 
                         if prereq in courses]
    if flattened_prereqs or course in courses:
        courses_with_prereqs[course] = flattened_prereqs
courses_with_prereqs

{'LIBS 150': [],
 'PACE 111T': [],
 'WRTG 111': [],
 'IFSM 201': [],
 'NUTR 100': [],
 'CMIT 202': [],
 'SPCH 100': [],
 'MATH 107': [],
 'WRTG 112': [],
 'CMIT 265': ['CMIT 202'],
 'HIST 125': [],
 'BIOL 103': [],
 'BEHS 103': [],
 'ARTH 334': [],
 'ELECTIVE 1': [],
 'ECON 103': [],
 'CMIT 291': ['CMIT 202', 'CMIT 265'],
 'CMIT 320': ['CMIT 265'],
 'ELECTIVE 2': [],
 'CMIT 321': ['CMIT 320'],
 'ELECTIVE 3': [],
 'CMIT 351': ['CMIT 265'],
 'ELECTIVE 4': [],
 'CMIT 326': [],
 'WRTG 393': ['WRTG 112'],
 'ELECTIVE 5': [],
 'ELECTIVE 6': [],
 'ELECTIVE 7': [],
 'CMIT 421': ['CMIT 320'],
 'ELECTIVE 8': [],
 'ELECTIVE 9': [],
 'CMIT 386': ['CMIT 321'],
 'ELECTIVE 10': [],
 'ELECTIVE 11': [],
 'CCJS 321': [],
 'ELECTIVE 12': [],
 'ELECTIVE 13': [],
 'ELECTIVE 14': [],
 'ELECTIVE 15': [],
 'CMIT 495': [],
 'CAPL 398A': []}

In [31]:
courses_with_prereqs = {}
for course, prereqs in courses.items():
    # Flatten the list of prerequisites and include only those present in the courses dictionary
    flattened_prereqs = [prereq for sublist in prereqs for prereq in sublist if prereq in courses]
    if flattened_prereqs or course in courses:
        courses_with_prereqs[course] = flattened_prereqs
courses_with_prereqs

{'LIBS 150': [],
 'PACE 111T': [],
 'WRTG 111': [],
 'IFSM 201': [],
 'NUTR 100': [],
 'CMIT 202': [],
 'SPCH 100': [],
 'MATH 107': [],
 'WRTG 112': [],
 'CMIT 265': ['CMIT 202'],
 'HIST 125': [],
 'BIOL 103': [],
 'BEHS 103': [],
 'ARTH 334': [],
 'ELECTIVE 1': [],
 'ECON 103': [],
 'CMIT 291': ['CMIT 202', 'CMIT 265'],
 'CMIT 320': ['CMIT 265'],
 'ELECTIVE 2': [],
 'CMIT 321': ['CMIT 320'],
 'ELECTIVE 3': [],
 'CMIT 351': ['CMIT 265'],
 'ELECTIVE 4': [],
 'CMIT 326': [],
 'WRTG 393': ['WRTG 112'],
 'ELECTIVE 5': [],
 'ELECTIVE 6': [],
 'ELECTIVE 7': [],
 'CMIT 421': ['CMIT 320'],
 'ELECTIVE 8': [],
 'ELECTIVE 9': [],
 'CMIT 386': ['CMIT 321'],
 'ELECTIVE 10': [],
 'ELECTIVE 11': [],
 'CCJS 321': [],
 'ELECTIVE 12': [],
 'ELECTIVE 13': [],
 'ELECTIVE 14': [],
 'ELECTIVE 15': [],
 'CMIT 495': [],
 'CAPL 398A': []}

In [29]:
all_prereqs = set()
for prereqs in courses.values():
    for sublist in prereqs:
        all_prereqs.update(sublist)
all_prereqs

{'CMIS 141',
 'CMIT 202',
 'CMIT 265',
 'CMIT 320',
 'CMIT 321',
 'CMSC 115',
 'WRTG 112'}

In [ ]:
import asyncio

course_order = asyncio.run(return_order(courses))
print(course_order)

In [26]:
df = await get_df_for_figure(timedConnection, program_id=10)
df = await rename_elective(df)
courses = await prep_df_for_new_order(df)
courses

{'LIBS 150': [],
 'PACE 111T': [],
 'WRTG 111': [],
 'IFSM 201': [],
 'NUTR 100': [],
 'CMIT 202': [],
 'SPCH 100': [],
 'MATH 107': [],
 'WRTG 112': [],
 'CMIT 265': [['CMIT 202'], ['CMSC 115'], ['CMIS 141']],
 'HIST 125': [],
 'BIOL 103': [],
 'BEHS 103': [],
 'ARTH 334': [],
 'ELECTIVE 1': [],
 'ECON 103': [],
 'CMIT 291': [['CMIT 202'], ['CMIT 265']],
 'CMIT 320': [['CMIT 265']],
 'ELECTIVE 2': [],
 'CMIT 321': [['CMIT 320']],
 'ELECTIVE 3': [],
 'CMIT 351': [['CMIT 265']],
 'ELECTIVE 4': [],
 'CMIT 326': [],
 'WRTG 393': [['WRTG 112']],
 'ELECTIVE 5': [],
 'ELECTIVE 6': [],
 'ELECTIVE 7': [],
 'CMIT 421': [['CMIT 320']],
 'ELECTIVE 8': [],
 'ELECTIVE 9': [],
 'CMIT 386': [['CMIT 321']],
 'ELECTIVE 10': [],
 'ELECTIVE 11': [],
 'CCJS 321': [],
 'ELECTIVE 12': [],
 'ELECTIVE 13': [],
 'ELECTIVE 14': [],
 'ELECTIVE 15': [],
 'CMIT 495': [],
 'CAPL 398A': []}

In [ ]:
course_order = await return_order(courses)
course_order

['LIBS 150',
 'PACE 111T',
 'WRTG 111',
 'IFSM 201',
 'NUTR 100',
 'CMIT 202',
 'SPCH 100',
 'MATH 107',
 'WRTG 112',
 'CMSC 115',
 'CMIS 141',
 'HIST 125',
 'BIOL 103',
 'BEHS 103',
 'ARTH 334',
 'ELECTIVE14',
 'ECON 103',
 'ELECTIVE18',
 'ELECTIVE20',
 'ELECTIVE22',
 'CMIT 326',
 'ELECTIVE25',
 'ELECTIVE26',
 'ELECTIVE27',
 'ELECTIVE29',
 'ELECTIVE30',
 'ELECTIVE32',
 'ELECTIVE33',
 'CCJS 321',
 'ELECTIVE35',
 'ELECTIVE36',
 'ELECTIVE37',
 'ELECTIVE38',
 'CMIT 495',
 'CAPL 398A',
 'WRTG 393',
 'CMIT 265',
 'CMIT 291',
 'CMIT 320',
 'CMIT 351',
 'CMIT 321',
 'CMIT 421',
 'CMIT 386']

In [ ]:
course_df[['pre']]

In [16]:
data = new_dict
# Convert to the desired format
courses = {}
for item in data:
    name = item['name']
    pre = item['pre']
    # Check if the course name ends with a star
    if name.endswith('*'):
        # If it does, remove the star
        name = name[:-1]
    if pre.endswith('*'):
        # If it does, remove the star
        pre = pre[:-1]
    if pre == '':
        courses[name] = []
    else:
        # Remove parentheses and split by '|'
        prerequisites = pre.strip('()').split(' | ')
        # Each prerequisite is a list of one item
        prerequisites = [[p.strip()] for p in prerequisites]
        courses[name] = prerequisites

courses

{'LIBS 150': [],
 'PACE 111T': [],
 'WRTG 111': [],
 'IFSM 201': [],
 'NUTR 100': [],
 'CMIT 202': [],
 'SPCH 100': [],
 'MATH 107': [],
 'WRTG 112': [],
 'CMIT 265': [['CMIT 202'], ['CMSC 115'], ['CMIS 141']],
 'HIST 125': [],
 'BIOL 103': [],
 'BEHS 103': [],
 'ARTH 334': [],
 'ELECTIVE': [],
 'ECON 103': [],
 'CMIT 291': [['CMIT 202'], ['CMIT 265']],
 'CMIT 320': [['CMIT 265']],
 'CMIT 321': [['CMIT 320']],
 'CMIT 351': [['CMIT 265']],
 'CMIT 326': [],
 'WRTG 393': [['WRTG 112']],
 'CMIT 421': [['CMIT 320']],
 'CMIT 386': [['CMIT 321']],
 'CCJS 321': [],
 'CMIT 495': [],
 'CAPL 398A': []}

In [21]:

from graphlib import TopologicalSorter

classes_flat = {course: [item for sublist in prereqs for item in sublist] for course, prereqs in my_classes.items()}
ts = TopologicalSorter(classes_flat)
order = list(ts.static_order())


In [19]:
my_classes = {
    "LIBS 150":  [],
    "PACE 111T": [],
    "WRTG 111":  [],
    "IFSM 201":  [],
    "NUTR 100":  [],
    "CMIT 202":  [],
    "SPCH 100":  [],
    "MATH 107":  [],
    "WRTG 112":  [],
    "CMIT 265":  [["CMIT 202"], ["CMSC 115"], ["CMIS 141"]], 
    "HIST 125":  [],
    "BIOL 103":  [],
    "BEHS 103":  [],
    "ARTH 334":  [],
    "ELECTIVE":  [],
    "ECON 103":  [],
    "CMIT 291":  [["CMIT 202"], ["CMIT 265"]],
    "CMIT 320":  [["CMIT 265"]],
    "CMIT 321":  [["CMIT 320"]],
    "CMIT 351":  [["CMIT 265"]],
    "CMIT 326":  [],
    "WRTG 393":  [["WRTG 112"]],
    "CMIT 421":  [["CMIT 320"]],
    "CMIT 386":  [["CMIT 321"]],
    "CCJS 321":  [],
    "CMIT 495":  [],
    "CAPL 398": []
}

In [22]:
order = list(ts.static_order())
order

['LIBS 150',
 'PACE 111T',
 'WRTG 111',
 'IFSM 201',
 'NUTR 100',
 'CMIT 202',
 'SPCH 100',
 'MATH 107',
 'WRTG 112',
 'CMSC 115',
 'CMIS 141',
 'HIST 125',
 'BIOL 103',
 'BEHS 103',
 'ARTH 334',
 'ELECTIVE',
 'ECON 103',
 'CMIT 326',
 'CCJS 321',
 'CMIT 495',
 'CAPL 398',
 'WRTG 393',
 'CMIT 265',
 'CMIT 291',
 'CMIT 320',
 'CMIT 351',
 'CMIT 321',
 'CMIT 421',
 'CMIT 386']

In [ ]:

period_idx = 1
period = periods_df.iloc[period_idx]
period

In [ ]:
def parse_prerequisites(prereq_string):
    '''
    Parse the prerequisites string and return a list of prerequisite groups.
    Each group contains lists of courses that satisfy the 'or' and 'and' patterns.
    '''
    import re
    prereq_string = prereq_string.strip()
    prereq_string = re.sub(r'\s+', ' ', prereq_string)
    prereq_string = re.sub(r'\(', '', prereq_string)
    prereq_string = re.sub(r'\)', '', prereq_string)
    
    and_groups = prereq_string.split('&')
    prereq_groups = [group.split('|') for group in and_groups]
    
    # Strip whitespace from each course in the groups
    prereq_groups = [[course.strip() for course in group] for group in prereq_groups]
    
    return prereq_groups

def eval_prerequisites(df, prereq_groups):
    '''
    Evaluate each prerequisite and return a parallel output with True or False values.
    True if the name was found in the DataFrame, and False if the name was not.
    '''
    import re
    
    eval_groups = []
    for pre_group in prereq_groups:
        eval_group = []
        for pre_course in pre_group:
            if '*' in pre_course:
                pre_course = pre_course.replace('*', '')  # Remove '*' from the course name
            if '+' in pre_course:
                course_prefix, course_number = re.match(r'(\D+)(\d+)\+', pre_course).groups()
                pre_rows = df[(df['name'].str.startswith(course_prefix)) & (df['name'].str.slice(start=len(course_prefix)).astype(int) >= int(course_number))]
            else:
                pre_rows = df[df['name'] == pre_course]
            eval_group.append(not pre_rows.empty)
        eval_groups.append(eval_group)
    return eval_groups


def prep_course_list(df):
    '''
    This function determines if all prerequisites are in the course list
    If so, it tries to arrange them so prerequisites are higher in the dataframe 
    than the course they are a prerequisite for.
    '''
    # Assuming df is your DataFrame and it has columns 'name', 'pre', 'met', and 'where'
    for i, row in df.iterrows():
        # only run on non-empty row['pre'] rows
        if pd.notna(row['pre']) and row['pre'] != '':
            pre_groups = parse_prerequisites(row['pre'])
            eval_groups = eval_prerequisites(df, pre_groups)
            # Note: eval groups evaluation:
            #       [[True, False, True], [True], [True, False]]
            met = False
            for j, pre_group in enumerate(pre_groups):
                if any(eval_groups[j]):  # If any prerequisite in the group is met
                    met = True
                    for k, pre_course in enumerate(pre_group):
                        if eval_groups[j][k]:  # If the prerequisite is met
                            pre_rows = df[df['name'] == pre_course]
                            if pre_rows.index[0] > i:
                                # Move the prerequisite row to immediately precede the current row
                                cols = df.columns.tolist()
                                temp = df.loc[pre_rows.index[0], cols].tolist()
                                df.loc[i+2:] = df.loc[i+1:-1].values  # Shift rows down
                                df.loc[i+1] = temp  # Insert the prerequisite row
                            break  # Exit the loop as soon as one prerequisite is met
                if met:
                    break  # Exit the loop as soon as one group of prerequisites is met
            df.at[i, 'met'] = met
            df.at[i, 'where'] = i - 1  # The prerequisite should be the previous row
    return df

In [ ]:
# debug for the first row of unlocked_df
i = 9
row = df.iloc[i]
print(f"Name: {row['name']}, Credits: {row['credits']}")
row['pre']

In [ ]:
pre_groups = parse_prerequisites(row['pre'])
eval_groups = eval_prerequisites(df, pre_groups)

In [ ]:
pre_groups

In [ ]:
eval_groups

In [ ]:
len(pre_groups)

In [ ]:
np.where(eval_groups[0])[0]

In [ ]:
# in this case, there's only one
for j, pre_group in enumerate(pre_groups):
    print(f'j: {j}, pre_group: {pre_group}')

In [ ]:
met = any(eval_groups[j])

In [ ]:
np.where(pre_group[0])

In [ ]:
pd.notna(row['pre'])

In [ ]:
def prep_course_list_older(df):
    '''
    This function takes a DataFrame as input and returns a modified DataFrame where the courses are 
    ordered based on their prerequisites. It handles both “or” and “and” conditions in prerequisites, 
    as well as the ‘*’ and ‘+’ notations.

    Please note that this function assumes that the update_prerequisites function returns a dictionary 
    with keys that match the columns of your DataFrame. 
    '''

    import re

    # Assuming df is the DataFrame and it has columns 'name', 'pre', 'met', and 'where'
    for i, row in df.iterrows():
        if pd.notna(row['pre']) and row['pre'] != '':
            pre_courses = [pre.split('&') for pre in row['pre'].split('|')]  # Split prerequisites on '|' and '&'
            met = False
            for pre_group in pre_courses:
                met_group = True
                for pre_course in pre_group:
                    pre_course = pre_course.strip()  # Remove leading/trailing whitespace
                    if '*' in pre_course:
                        pre_course = pre_course.replace('*', '')  # Remove '*' from the course name
                    if '+' in pre_course:
                        course_prefix, course_number = re.match(r'(\D+)(\d+)\+', pre_course).groups()
                        pre_rows = df[(df['name'].str.startswith(course_prefix)) & (df['name'].str.slice(start=len(course_prefix)).astype(int) >= int(course_number))]
                    else:
                        pre_rows = df[df['name'] == pre_course]
                    if not pre_rows.empty:
                        if pre_rows.index[0] > i:
                            # Move the prerequisite row to immediately precede the current row
                            cols = df.columns.tolist()
                            temp = df.loc[pre_rows.index[0], cols].tolist()
                            df.loc[i+2:] = df.loc[i+1:-1].values  # Shift rows down
                            df.loc[i+1] = temp  # Insert the prerequisite row
                    else:
                        # If the prerequisite is not found in the DataFrame, get it and insert it
                        print(f'The prerequisite {pre_course} is not found, it should be inserted!')
                        #insert_prerequisite(df, i, pre_course)
                        met_group = False
                        break  # Exit the loop as soon as one prerequisite is not met
                if met_group:
                    met = True
                    break  # Exit the loop as soon as one group of prerequisites is met
            df.at[i, 'met'] = met
            df.at[i, 'where'] = i - 1  # The prerequisite should be the previous row
    return df


In [ ]:
def prep_course_list_2(df):
    '''
    This function takes a DataFrame as input and returns a modified DataFrame where the courses are 
    ordered based on their prerequisites. It handles both “or” and “and” conditions in prerequisites, 
    as well as the ‘*’ and ‘+’ notations.

    Please note that this function assumes that the update_prerequisites function returns a dictionary 
    with keys that match the columns of your DataFrame. 
    '''

    import re

    # Assuming df is the DataFrame and it has columns 'name', 'pre', 'met', and 'where'
    for i, row in df.iterrows():
        if pd.notna(row['pre']) and row['pre'] != '':
            pre_courses = [pre.split('&') for pre in row['pre'].split('|')]  # Split prerequisites on '|' and '&'
            met = False
            for pre_group in pre_courses:
                met_group = True
                for pre_course in pre_group:
                    pre_course = pre_course.strip()  # Remove leading/trailing whitespace
                    if '*' in pre_course:
                        pre_course = pre_course.replace('*', '')  # Remove '*' from the course name
                    if '+' in pre_course:
                        course_prefix, course_number = re.match(r'(\D+)(\d+)\+', pre_course).groups()
                        pre_rows = df[(df['name'].str.startswith(course_prefix)) & (df['name'].str.slice(start=len(course_prefix)).astype(int) >= int(course_number))]
                    else:
                        pre_rows = df[df['name'] == pre_course]
                    if not pre_rows.empty:
                        if pre_rows.index[0] > i:
                            # Move the prerequisite row to immediately precede the current row
                            cols = df.columns.tolist()
                            temp = df.loc[pre_rows.index[0], cols].tolist()
                            df.loc[i+2:] = df.loc[i+1:-1].values  # Shift rows down
                            df.loc[i+1] = temp  # Insert the prerequisite row
                    else:
                        # If the prerequisite is not found in the DataFrame, get it and insert it
                        print(f'The prerequisite {pre_course} is not found, it should be inserted!')
                        #insert_prerequisite(df, i, pre_course)
                        met_group = False
                        break  # Exit the loop as soon as one prerequisite is not met
                if met_group:
                    met = True
                    break  # Exit the loop as soon as one group of prerequisites is met
            df.at[i, 'met'] = met
            df.at[i, 'where'] = i - 1  # The prerequisite should be the previous row
    return df


In [ ]:
import re

def tokenize(expression):
    import re

    # Define the token pattern, accommodating course names
    token_pattern = r'\s*(\(|\)|\&|\||[A-Z]+\s+\d{3})\s*'
    tokens = re.findall(token_pattern, expression)
    return [token for token in tokens if token]

#def parse(tokens):
#    import re
#
#    def parse_expression(index):
#        if tokens[index] == '(':
#            sub_expressions = []
#            index += 1
#            while tokens[index] != ')':
#                if tokens[index] in ('&', '|'):
#                    operator = tokens[index]
#                    index += 1
#                else:
#                    operand, index = parse_expression(index)
#                    sub_expressions.append(operand)
#            index += 1
#            if operator == '&':
#                return ['and'] + sub_expressions, index
#            elif operator == '|':
#                return ['or'] + sub_expressions, index
#        elif re.match(r'[A-Z]+\s+\d{3}', tokens[index]):
#            return f"match('{tokens[index]}')", index + 1
#    
#    expression_tree, _ = parse_expression(0)
#    return expression_tree

def parse_boolean_expression(expression):
    tokens = tokenize(expression)
    parsed_expression = parse(tokens)
    # Convert string 'match' calls to actual function calls
    parsed_expression = eval(parsed_expression)
    return parsed_expression


In [ ]:

# Example usage
expression = "(CMIT 202 | CMSC 115 | CMIS 141)"
parse_boolean_expression(expression)



In [ ]:
parsed_expression = parse(tokens)
parsed_expression

In [ ]:
parsed_expression = parse_boolean_expression(expression)


In [ ]:

print(parsed_expression)  # Output: ['or', match('CMIT 202'), match('CMSC 115'), match('CMIS 141')]


In [ ]:

# Now evaluate the parsed expression using the previous evaluate function
result = evaluate(parsed_expression)
print(result)  # Output: The boolean result based on match function


In [ ]:
parse_prerequisites('((STAT 200 | MATH 400) & ARTS 100)')



testcase = parse_prerequisites('(CMIT 202 | CMSC 115 | CMIS 141)')
testcase
two_ands = parse_prerequisites('((STAT 200 | MATH 400) & ARTS 100)')



In [ ]:
print(f'The length of {two_ands} is {len(two_ands)}')
print(f'')

In [ ]:
import re

def insert_prerequisite(df, i, name):
    # Recursive function to insert prerequisites
    pre_dict = get_prerequisite(name)
    if pre_dict is not None:
        # Shift rows down
        df.loc[i+2:] = df.loc[i+1:-1].values
        # Insert the prerequisite row
        df.loc[i+1] = pre_dict
        # Check if the inserted prerequisite has its own prerequisites
        insert_prerequisite(df, i+1, pre_dict['name'])


In [ ]:
def prep_course_list_3(df):
    # Assuming df is your DataFrame and it has columns 'name', 'pre', 'met', and 'where'
    for i, row in df.iterrows():
        if pd.notna(row['pre']) and row['pre'] != '':
            pre_groups = parse_prerequisites(row['pre'])
            met = False
            for pre_group in pre_groups:
                met_group = True
                for pre_course in pre_group:
                    if '*' in pre_course:
                        pre_course = pre_course.replace('*', '')  # Remove '*' from the course name
                    if '+' in pre_course:
                        course_prefix, course_number = re.match(r'(\D+)(\d+)\+', pre_course).groups()
                        pre_rows = df[(df['name'].str.startswith(course_prefix)) & (df['name'].str.slice(start=len(course_prefix)).astype(int) >= int(course_number))]
                    else:
                        pre_rows = df[df['name'] == pre_course]
                    if not pre_rows.empty:
                        if pre_rows.index[0] > i:
                            # Move the prerequisite row to immediately precede the current row
                            cols = df.columns.tolist()
                            temp = df.loc[pre_rows.index[0], cols].tolist()
                            df.loc[i+2:] = df.loc[i+1:-1].values  # Shift rows down
                            df.loc[i+1] = temp  # Insert the prerequisite row
                    else:
                        # If the prerequisite is not found in the DataFrame, get it and insert it
                        print(f'The prerequisite {pre_course} is not found, it should be inserted!')
                        #insert_prerequisite(df, i, pre_course)
                        met_group = False
                        break  # Exit the loop as soon as one prerequisite is not met
                if met_group:
                    met = True
                    break  # Exit the loop as soon as one group of prerequisites is met
            df.at[i, 'met'] = met
            df.at[i, 'where'] = i - 1  # The prerequisite should be the previous row
    return df

def prep_course_list_4(df):
    # Assuming df is your DataFrame and it has columns 'name', 'pre', 'met', and 'where'
    for i, row in df.iterrows():
        if pd.notna(row['pre']) and row['pre'] != '':
            pre_groups = parse_prerequisites(row['pre'])
            met = False
            for pre_group in pre_groups:
                met_group = False
                for pre_course in pre_group:
                    if '*' in pre_course:
                        pre_course = pre_course.replace('*', '')  # Remove '*' from the course name
                    if '+' in pre_course:
                        course_prefix, course_number = re.match(r'(\D+)(\d+)\+', pre_course).groups()
                        pre_rows = df[(df['name'].str.startswith(course_prefix)) & \
                                      (df['name'].str.slice(start=len(course_prefix)).astype(int) >= int(course_number))]
                    else:
                        pre_rows = df[df['name'] == pre_course]
                    if not pre_rows.empty:
                        if pre_rows.index[0] > i:
                            # Move the prerequisite row to immediately precede the current row
                            cols = df.columns.tolist()
                            temp = df.loc[pre_rows.index[0], cols].tolist()
                            df.loc[i+2:] = df.loc[i+1:-1].values  # Shift rows down
                            df.loc[i+1] = temp  # Insert the prerequisite row
                        met_group = True
                        break  # Exit the loop as soon as one prerequisite is met
                if met_group:
                    met = True
                    break  # Exit the loop as soon as one group of prerequisites is met
            df.at[i, 'met'] = met
            df.at[i, 'where'] = i - 1  # The prerequisite should be the previous row
    return df


In [ ]:
def prep_course_list(df):
    # Assuming df is your DataFrame and it has columns 'name', 'pre', 'met', and 'where'
    for i, row in df.iterrows():
        if pd.notna(row['pre']) and row['pre'] != '':
            pre_groups = parse_prerequisites(row['pre'])
            eval_groups = eval_prerequisites(df, pre_groups)
            met = False
            for j, pre_group in enumerate(pre_groups):
                if any(eval_groups[j]):  # If any prerequisite in the group is met
                    met = True
                    for k, pre_course in enumerate(pre_group):
                        if eval_groups[j][k]:  # If the prerequisite is met
                            pre_rows = df[df['name'] == pre_course]
                            if pre_rows.index[0] > i:
                                # Move the prerequisite row to immediately precede the current row
                                cols = df.columns.tolist()
                                temp = df.loc[pre_rows.index[0], cols].tolist()
                                df.loc[i+2:] = df.loc[i+1:-1].values  # Shift rows down
                                df.loc[i+1] = temp  # Insert the prerequisite row
                            break  # Exit the loop as soon as one prerequisite is met
                if met:
                    break  # Exit the loop as soon as one group of prerequisites is met
            df.at[i, 'met'] = met
            df.at[i, 'where'] = i - 1  # The prerequisite should be the previous row
    return df

In [ ]:
df.head()


In [ ]:
classes = {
    "A": [],
    "B": [],
    "C": [],
    "D": [],
    "E": [["A", "C"]],  # E depends on A and C
    "F": [],
    "G": [],
    "H": ["D"],
    "I": [],
    "J": [["H"], ["F"]]  # J depends on H or F
}


In [ ]:
#result = utils.prep_course_list(df)
result = prep_course_list(df)


#course_df.head()

In [ ]:
prereq_df = course_df[course_df['pre'] != ''][['name', 'pre']]
prereq_dict = prereq_df.to_dict(orient='records')

prereq_dict

In [ ]:
prereqs = {row['name']: row for row in prereq_dict}
for key, data in prereqs.items():
    prereqs[key]['grouped'] = utils.parse_prerequisites(prereqs[key]['pre'])

In [ ]:
key = 'CMIT 265'
prereqs[key]['grouped']

In [ ]:
after = await utils.handle_prerequisites(timedConnection, prereq_df)
after

### Testing the generate_schedule function

In [ ]:
# So we see the entire df
pd.set_option('display.max_rows', None)

In [ ]:
schedule = []

course_df = df.copy()
locked_df   = course_df[course_df['locked']]
unlocked_df = course_df[~course_df['locked']]


In [ ]:
# debug for the first row of unlocked_df
irow = 0
course = unlocked_df.iloc[irow]
print(f"Name: {course['name']}, Credits: {course['credits']}")

period_idx = 1
period = periods_df.iloc[period_idx]
period

In [ ]:
periods_df.at[period_idx, 'max_courses'] > 0

In [ ]:
# get the row number where w is its location
row = max_credits_df.loc[max_credits_df['period'] == period['period']]
w = row.index.to_list()[0]
max_credits = max_credits_df.at[w, 'max_credits']
max_credits

In [ ]:
schedule = []

In [ ]:
if (periods_df.at[period_idx, 'max_courses'] > 0) and (max_credits >= course['credits']):
    # there is room to schedule this class
    # need to check later that it has no prerequisites

    schedule.append({
        'seq': len(schedule) + 1,
        'name': course['name'],
        'course_type': course['course_type'],
        'type': course['type'],
        'credits': int(course['credits']),
        'title': course['title'],
        'term': period['term'],
        'year': period['year'],
        'session': period['session'],
        'period': period['period'],
        'pre': course['pre'],
        'pre_credits': course['pre_credits'],
        'substitutions': course['substitutions'],
        'description': course['description'],
        'locked': False
    })

    # keep track of credits and courses remaining for this term-year-session
    max_credits_df.at[w, 'max_credits'] -= int(course['credits'])
    periods_df.at[period_idx, 'max_courses'] -= 1
    unlocked_df.at[irow, 'scheduled'] = True

schedule

In [ ]:
# check if there are classes remaining
periods_df.at[period_idx, 'max_courses'] > 0

In [ ]:

periods_df.head()

In [ ]:
del periods_df['max_credits']

In [ ]:
max_credits_df.head()

In [ ]:
course_df = df.copy()
course_df.head()

In [ ]:
course_df[['seq', 'name', 'credits', 'completed', 'term', 'session', 'locked', 'pre']]

In [ ]:

schedule_df = await utils.generate_schedule(timedConnection, course_df, periods_df)

In [ ]:
schedule_df

## Debug generate_schedule here



In [ ]:
def handle_specific_row(df, row_index):
    '''
    Handle a specific row by index, similar to how iterrows handles each row.
    '''
    if row_index >= 0 and row_index < len(df):
        course = df.iloc[row_index]
        # Now `course` is a Series representing the row at `row_index`
        # Process the course row as needed
        print(f"Handling row at index {row_index}")
        print(course)
    else:
        print("Index out of bounds")

# Example usage
handle_specific_row(df, 0)  # Handling the first row

In [ ]:
assigned = False
period_idx = 0
period = periods_df.iloc[period_idx]
period

In [ ]:
term_year = (period['term'], period['year'])
term_year

In [ ]:
## Test whether the previous period is within scope of the schedule
## E.g., a course with a prerequisite cannot be scheduled in the first period
previous_period_idx = period_idx - period['previous']
if previous_period_idx >= 0:
    has_previous = True
    previous_period = periods_df.iloc[previous_period_idx]
else:
    has_previous = False

print(f'has_previous: {has_previous}, previous_period_idx: {previous_period_idx}')

In [ ]:
periods_df.at[period_idx, 'max_courses']

In [ ]:
course_df[['term', 'year']]

In [ ]:
default_first_term = 'Spring 2024'
disabled_program_menu_items = {
        'Cybersecurity Technology',
        'Social Science',
        'Applied Technology',
        'Web and Digital Design',        
        'East Asian Studies',
        'English',
        'General Studies',
        'History'
    }
role = 'guest'
logged_in = False

## Testing the create schedule from scratch capabilities

In [ ]:
# Settled Functions

# parallel the utils.get_required_program_courses code
async def get_required_program_courses_no_q(timedConnection, student_info):
    '''
    DELETE after testing. Use the get_required_program_courses code instead
    '''
    program_id = student_info['program_id']
    query = '''
        SELECT 
            id,
            course, 
            course_type as type,
            title,
            credits,
            pre,
            pre_credits,
            substitutions,
            description
        FROM program_requirements_view
        WHERE program_id = ?
    '''
    df = await get_query_df(timedConnection, query, params=(program_id,))
    return df


In [ ]:
# Once program is chosen, fill in GE information including any GE 
await utils.return_program_course_list_df_from_scratch(timedConnection, student_info)

In [ ]:
# Build the Program course list

# Task 1. Get the required classes from the program
required_df = await get_required_program_courses_no_q(timedConnection, student_info)

# Build the GE course list
# Create a blank list of GE course requirements with all courses set to 'GENERAL'
query = '''
    SELECT id AS ge_id, 'GENERAL' AS course, requirement, abbr, part, credits, 
        'ge_' || abbr || '_' || part AS course_slot 
    FROM general_education_requirements 
'''
ge_course_list = await get_query_dict(timedConnection, query)

# update with information from student_info['ge']
ge_course_list = utils.update_ge_list_from_student_info(ge_course_list, student_info)

# Update the ge_course_list 
ge_course_list, elective_data = await utils.build_program_course_list(required_df, timedConnection, ge_course_list)

# check on the elective_data as well (it should be empty)

# Process ge_course_list_df
ge_course_list_df = pd.DataFrame(ge_course_list)
ge_course_list_df = utils.update_bio_df(ge_course_list_df)
ge_course_list_df['type'] = 'General'
ge_course_list_df = ge_course_list_df.rename(columns={'course_slot': 'ge'})

# required_course_list_df is where we keep all of our courses for now
required_course_list_df = required_df[['course', 'type', 'credits']].copy()

# Merge GE information onto any 'Required,General' rows )
merged_df = pd.merge(ge_course_list_df[['course', 'ge']], required_course_list_df, 
                     on='course', how='right')

# remove from the GE list those already in the Program list
remove_list = merged_df[merged_df['type']=='Required,General'][['ge']]
if len(remove_list) > 0:
    remove_list_values = remove_list['ge'].values.tolist()
    mask = ~ge_course_list_df['ge'].isin(remove_list_values)
    ge_course_list_df = ge_course_list_df[mask][['course', 'ge', 'type', 'credits']]
else:
    ge_course_list_df = ge_course_list_df[['course', 'ge', 'type', 'credits']]

# concatenate lists to join all 'General' to course list
# this will work either way
course_list_df = pd.concat([merged_df, ge_course_list_df], ignore_index=True)
course_list_df['credits'] = pd.to_numeric(course_list_df['credits'])

course_list_df['ge'] = course_list_df['ge'].fillna('')
course_list_df
# Add Electives to our course_list_df:

elective_credits = 120 - course_list_df['credits'].sum()

# Calculate number of electives to add 
# (standard pattern is one 1-credit class and the rest 3-credit classes)
# (1-credit class is often a capstone seminar)
# We will calculate them instead. 

remaining_elective_credit = elective_credits % 3
r = int(remaining_elective_credit)
number_of_electives = (elective_credits - remaining_elective_credit)/3
n = int(number_of_electives)

# create a DataFrame with the desired rows
data = {
    'course': ['ELECTIVE'] * (n+1),
    'ge': [''] * (n+1),
    'type': ['Elective'] * (n+1),
    'credits': [3] * n + [r]
}
elective_df = pd.DataFrame(data)

# Add electives to final dataframe

final_course_list = pd.concat([course_list_df, elective_df], ignore_index=True)

In [ ]:
final_course_list['credits'].sum()

In [ ]:
df1 = required_course_list_df
df2 = ge_course_list_df

In [ ]:
# Process ge_course_list_df
ge_course_list_df = pd.DataFrame(ge_course_list)
ge_course_list_df = utils.update_bio_df(ge_course_list_df)
ge_course_list_df['type'] = 'General'
ge_course_list_df = ge_course_list_df.rename(columns={'course_slot': 'ge'})

# required_course_list_df is where we keep all of our courses for now
required_course_list_df = required_df[['id', 'course', 'type', 'credits']].copy()

# Merge possible GE (should only work if there are any type = 'Required,General' )
merged_df = pd.merge(required_course_list_df, ge_course_list_df[['ge_id', 'course', 'ge']], on='course')

if len(merged_df) > 0:

    # concatenate original and 'Required,General' merged data
    course_list_df = pd.concat(
        [course_list_df[course_list_df['type'] != 'Required,General'], merged_df], ignore_index=True)
    
    # removed merged_df components from ge_course_list_df
    ge_list_df = ge_course_list_df[ge_course_list_df ]



In [ ]:
course_list_df['ge_id'] = np.nan
course_list_df['ge'] = ''
if len(merged_df) > 0:
    # Remove those rows from required_df
    

In [ ]:
df2 = ge_course_list_df[['course', 'type', 'credits', 'ge']].copy()
# Remove rows from df2 that have courses already in df1
df2 = df2[~df2['course'].isin(df1['course'])]
df2

In [ ]:
ge_course_list_df[~ge_course_list_df['ge_id'].isin(merged_df['ge_id'].unique())][['course', 'type', 'credits', 'ge_id', 'ge']]

In [ ]:
filtered_df = program_df[program_df['type'] != 'Required,General'][['id','course','type', 'credits']]
filtered_df['ge_id'] = np.nan
filtered_df['ge'] = ''
filtered_df

In [ ]:

df2 = ge_course_list_df[['course', 'type', 'credits', 'ge']].copy

#df2_filtered = df2[~df2['course'].isin(df1['course'])]
#df2_filtered

In [ ]:
df2['course']

In [ ]:
# Step 1: Filter df to get bio_df
bio_df = df[(df['course'] == 'GENERAL') & (df['course_slot'].str.contains('bio_1[ab]'))]
bio_df


In [ ]:
len(bio_df)

In [ ]:
first_row = bio_df.iloc[0]
df.loc[df['ge_id'] == first_row['ge_id'], 'part'] = '1'
df.loc[df['ge_id'] == first_row['ge_id'], 'course_slot'] = 'ge_bio_1'


In [ ]:
df

In [ ]:
df = df[~df['ge_id'].isin(bio_df['ge_id'])]
df

In [ ]:
df.loc[df['part'].str.contains('1[abc]'), 'part'] = '1'
df.loc[df['course_slot'].str.contains('ge_bio_1[abc]'), 'course_slot'] = 'ge_bio_1'
df

In [ ]:
# Step 2: Check the length of bio_df
if len(bio_df) == 3:
    # Take the first row and update 'part' and 'course_slot' in the original df
    first_row = bio_df.iloc[0]
    df.loc[df['ge_id'] == first_row['ge_id'], 'part'] = '1'
    df.loc[df['ge_id'] == first_row['ge_id'], 'course_slot'] = 'ge_bio_1'
    
    # Remove the other two rows from df
    df = df[~df['ge_id'].isin(bio_df.iloc[1:]['ge_id'])]
    
elif len(bio_df) == 2:
    # Delete rows from df identified by their 'ge_id'
    df = df[~df['ge_id'].isin(bio_df['ge_id'])]
else:
    # this should raise an error
    # add this later
    pass

# Step 3: Change 'part' and 'course_slot' in df
df.loc[df['part'].str.contains('1[abc]'), 'part'] = '1'
df.loc[df['course_slot'].str.contains('ge_bio_1[abc]'), 'course_slot'] = 'ge_bio_1'


In [ ]:
count_done_bio = len(df[(df['course'] != 'GENERAL') & (df['course_slot'].str.contains('bio_1'))])
count_done_bio

# if count_done_bio = 0, then delete 

In [ ]:
# Check if any of the abbr='bio' and part='1[abc]' are not 'GENERAL'
bio_non_general = any(course['course'] != 'GENERAL' for course in ge_remaining_list if course['abbr'] == 'bio' and course['part'] in ['1a', '1b', '1c'])
bio_non_general

In [ ]:

# Filter out the records with abbr='bio' and part='1[abc]'
bio_records = [course for course in ge_course_list if course['abbr'] == 'bio' and course['part'] in ['1a', '1b', '1c']]

# Check if all three parts are 'GENERAL'
all_general = all(course['course'] == 'GENERAL' for course in bio_records)

# If all three parts are 'GENERAL', rename part='1a' to part='1' and omit '1b' and '1c'
if all_general:
    # Rename part='1a' to part='1'
    for course in ge_course_list:
        if course['abbr'] == 'bio' and course['part'] == '1a':
            course['part'] = '1'
    # Filter out '1b' and '1c' records
    ge_course_list = [course for course in ge_course_list if not (course['abbr'] == 'bio' and course['part'] in ['1b', '1c'])]
else:
    # If one of the parts is not 'GENERAL', find the non-'GENERAL' part and rename it to '1'
    for course in bio_records:
        if course['course'] != 'GENERAL':
            course['part'] = '1'
            # Filter out '1a', '1b', and '1c' records except the non-'GENERAL' part
            ge_course_list = [c for c in ge_course_list if not (c['abbr'] == 'bio' and c['part'] != course['part'])]

# Filter out the records with course='GENERAL'
general_courses = [course for course in ge_course_list if course['course'] == 'GENERAL']


In [ ]:
pd.DataFrame(ge_course_list)
df[df['course'] == 'GENERAL']

In [ ]:
ge_course_list

In [ ]:
row = ge_course_list[2]
row

In [ ]:
pd.DataFrame(ge_course_list)

In [ ]:
student_info = update_student_info_ge(student_info, ge_course_list)
student_info

In [ ]:
# Now fill in the rest of the GE requirements
student_info['ge']

In [ ]:
program_query = '''
    SELECT program_id AS name, program_name AS label, disabled
    FROM menu_all_view 
    WHERE menu_degree_id = ? AND menu_area_id = ?
'''

In [ ]:
rows = await get_query(timedConnection, program_query, (1, 7))
[(row['name'], row['label'], bool(row['disabled']) ) for row in rows]

In [ ]:
choices = await utils.get_choices_with_disabled(timedConnection, program_query, (1, 7))
choices

In [ ]:
# replicate code from get_required_program_courses(q)
async def get_required_df(program_id, timedConnection = conn):
    query = '''
        SELECT 
            id,
            course, 
            course_type as type,
            title,
            credits,
            pre,
            pre_credits,
            substitutions,
            description
        FROM program_requirements_view
        WHERE program_id = ?
    '''
    df = await get_query_df(timedConnection, query, params=(program_id,))
    return df

program_id = 2
df = await get_required_df(program_id)
df.head()

df[df['type'] == 'Required,General']

# Step 0: Convert required df to a dictionary
dict_list = df.to_dict(orient='records')

# Step 1: Extract the required courses
required_courses = [row['course'] for row in dict_list if row['type'] == 'Required,General']
required = tuple(required_courses)
required

# Step 2: Construct the SQL query
# Using placeholders for the IN clause
query = f'''
    SELECT ge_id, course 
    FROM ge_view 
    WHERE course IN ({','.join(['?' for _ in required_courses])})
'''

# Step 3: Execute the query
result = await get_query_dict(timedConnection, query, params=required)
result

In [ ]:
result

In [ ]:
#old_id: new_id
ge_map = [
    { 'old_id': 14, 'new_id': 16, 'abbr': 'res'},
    { 'old_id': 13, 'new_id': 15, 'abbr': 'res'},
    { 'old_id': 12, 'new_id': 14, 'abbr': 'res'},
    { 'old_id': 11, 'new_id': 13, 'abbr': 'beh'},
    { 'old_id': 11, 'new_id': 12, 'abbr': 'beh'},
    { 'old_id': 10, 'new_id': 11, 'abbr': 'bio'},
    { 'old_id':  9, 'new_id': 10, 'abbr': 'bio'},
    { 'old_id':  8, 'new_id':  9, 'abbr': 'bio'},
    { 'old_id':  7, 'new_id':  8, 'abbr': 'bio'},
    { 'old_id':  6, 'new_id':  7, 'abbr': 'arts'},
    { 'old_id':  6, 'new_id':  6, 'abbr': 'arts'},
    { 'old_id':  5, 'new_id':  5, 'abbr': 'math'},
    { 'old_id':  4, 'new_id':  4, 'abbr': 'comm'},
    { 'old_id':  3, 'new_id':  3, 'abbr': 'comm'},
    { 'old_id':  2, 'new_id':  2, 'abbr': 'comm'},
    { 'old_id':  1, 'new_id':  1, 'abbr': 'comm'}
    ]

In [ ]:
dict_list

In [ ]:
courses = [{'course': row['course'], 'credits': row['credits'], 
            'ge': '', 'type': row['type']}  for row in dict_list]
courses

In [ ]:



req_ge = [row['course'] for row in courses if row['type'] == 'Required,General']
req_ge

SELECT ge_id, course
FROM ge_view 
WHERE course IN ('ECON 201', 'ECON 203', 'IFSM 300', 'STAT 200')


In [ ]:
student_info = utils.initialize_student_info()
student_info

In [ ]:
student_data = utils.initialize_student_data()
student_data

In [ ]:
user_id = 3
student_info['user_id'] = user_id
student_data['user_id'] = user_id

In [ ]:
query = 'SELECT * FROM student_info_view WHERE user_id = ?'
row = await get_query_one(timedConnection, query, params=(user_id,))


In [ ]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
              'transfer_credits', 'program_id']
student_info.update({name: row[name] for name in attributes})
student_info

In [ ]:
row = await utils.get_program_title(timedConnection, student_info['program_id'])
if row:
    student_info['degree_program'] = row['title']
    student_info['degree_id'] = row['id']
student_info

In [ ]:
student_data

In [ ]:
query = 'SELECT * FROM student_progress_d3_view WHERE user_id = ?'
# note: 'course' is named 'name' in student_progress_d3_view 
df = await get_query_df(conn, query, params=(student_info['user_id'],))
student_data['schedule'] = df

In [ ]:
html_template = utils.create_html_template(df, 'SPRING 2024')


In [ ]:
df_input = df.copy()
df_input.rename(columns={'term': 'period'}, inplace=True)
df_display, headers_display = utils.prepare_d3_data(df_input)

In [ ]:
df_display

In [ ]:
periods = utils.generate_periods(summer=True)


In [ ]:
def parse_prerequisites(prereq_string):
    '''
    Parse the prerequisites string and return a list of prerequisite groups.
    Each group contains lists of courses that satisfy the 'or' and 'and' patterns.
    '''
    import re
    prereq_string = prereq_string.strip()
    prereq_string = re.sub(r'\s+', ' ', prereq_string)
    prereq_string = re.sub(r'\(', '', prereq_string)
    prereq_string = re.sub(r'\)', '', prereq_string)
    
    and_groups = prereq_string.split('&')
    prereq_groups = [group.split('|') for group in and_groups]
    
    # Strip whitespace from each course in the groups
    prereq_groups = [[course.strip() for course in group] for group in prereq_groups]
    
    return prereq_groups


In [ ]:
parse_prerequisites('(CMIT 202 | CMIT 265)')

In [ ]:
parse_prerequisites('(STAT 200 & (MATH 115 | MATH 108))')

# Keycloak Stuff

In [ ]:
dir(keycloak)

In [ ]:
import keycloak

In [ ]:
#from keycloak import KeycloakOpenID
#from keycloak.exceptions import KeycloakAuthenticationError

In [ ]:
keycloak

In [ ]:
# Configuration
keycloak_server_url = "http://localhost:8080/auth/"
keycloak_realm_name = "your-realm"
keycloak_client_id = "your-client-id"
keycloak_client_secret = "your-client-secret"  # If using a confidential client
keycloak_redirect_uri = "http://localhost:8888/callback"
keycloak_username = "sample-user"
keycloak_password = "sample-password"

# Initialize Keycloak client
keycloak_openid = KeycloakOpenID(server_url=keycloak_server_url,
                                 client_id=keycloak_client_id,
                                 realm_name=keycloak_realm_name,
                                 client_secret_key=keycloak_client_secret)

# Authenticate and obtain access token
try:
    token = keycloak_openid.token(keycloak_username, keycloak_password)
    access_token = token['access_token']
    print("Access token:", access_token)
except KeycloakAuthenticationError as e:
    print(f"Failed to authenticate: {e}")

# Fetch user information
if access_token:
    userinfo = keycloak_openid.userinfo(access_token)
    print("User information:", userinfo)

In [ ]:
'  My Name'.strip()

In [ ]:
timedConnection = conn
program_id = 5
user_id = 3
student_data = {}

In [ ]:
query = 'SELECT * FROM student_progress_d3_view WHERE user_id = ?'
df = await get_query_df(timedConnection, query, params=(user_id,))
student_data['schedule'] = df
df

In [ ]:
query = '''
        SELECT 
            id,
            course, 
            course_type as type,
            title,
            credits,
            pre,
            pre_credits,
            substitutions,
            description
        FROM program_requirements_view
        WHERE program_id = ?
'''
df = await get_query_df(timedConnection, query, params=(program_id,))
student_data['required'] = df
df

In [ ]:
student_data

In [ ]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
        'transfer_credits', 'program_id']

In [ ]:
attributes

In [ ]:
student_info

In [ ]:
aa = 'ge_bio_1a'

In [ ]:
aa.split('_')

In [ ]:
aa.split('_')[2]

In [44]:
df = await get_df_for_figure(timedConnection, 10)
df.head()

,program_id,seq,name,course_type,type,credits,title,completed,term,year,session,locked,pre,pre_credits,substitutions,description,scheduled
0,10,1,LIBS 150,general,general,1,Introduction to Research,0,0,0,0,False,,,"COMP 111, LIBS 100, or LIBS 150",An introduction to the research process and me...,False
1,10,2,PACE 111T,general,general,3,Program and Career Exploration in Technology,0,0,0,0,False,,,"PACE 111B, PACE 111C, PACE 111M, PACE 111P, PA...",(Fulfills the general education requirement in...,False
2,10,3,WRTG 111,general,general,3,Academic Writing I,0,0,0,0,False,,,"WRTG 100A, WRTG 111, or WRTG 111X",(The first course in the two-course series WRT...,False
3,10,4,IFSM 201,general,general,3,Concepts and Applications of Information Techn...,0,0,0,0,False,,,"BMGT 301, CAPP 101, CAPP 300, CMST 300, IFSM 2...",(Access to a standard office productivity pack...,False
4,10,5,NUTR 100,general,general,3,Elements of Nutrition,0,0,0,0,False,,,NUTR 100 or NUTR 200,A study of the scientific and quantitative fou...,False


In [45]:
periods_df = utils.generate_periods(
    summer=False,
)
#    #start_term='student_info['first_term']',
#    start_term='SPRING 2024',
#    max_courses=schedule['courses_per_session'],
#    max_credits=schedule['max_credits'],
#    summer=schedule['attend_summer'],
#    sessions=schedule['sessions'],
#    as_df=True
#)
periods_df
periods_df['period'] = periods_df['term'] + ' ' + periods_df['year'].astype(str)
periods_df

,id,term,session,year,max_courses,max_credits,previous,period
0,1,SPRING,1,2024,3,18,1,SPRING 2024
1,2,SPRING,2,2024,0,0,2,SPRING 2024
2,3,SPRING,3,2024,3,18,2,SPRING 2024
3,4,SUMMER,1,2024,0,0,1,SUMMER 2024
4,5,SUMMER,2,2024,0,0,2,SUMMER 2024
...,...,...,...,...,...,...,...,...
80,81,SUMMER,1,2031,0,0,1,SUMMER 2031
81,82,SUMMER,2,2031,0,0,2,SUMMER 2031
82,83,FALL,1,2031,3,18,1,FALL 2031
83,84,FALL,2,2031,0,0,2,FALL 2031


In [46]:
# Keep track of max credits per (term, year), e.g., 'SPRING 2024',
# since credit constraints are officially managed at that level rather
# than as the (term, year, session) level

max_credits_df = periods_df.groupby(['year', 'term', 'period'])['max_credits'].max().reset_index()
del periods_df['max_credits']

In [47]:
max_credits_df

,year,term,period,max_credits
0,2024,FALL,FALL 2024,18
1,2024,SPRING,SPRING 2024,18
2,2024,SUMMER,SUMMER 2024,0
3,2025,FALL,FALL 2025,18
4,2025,SPRING,SPRING 2025,18
5,2025,SUMMER,SUMMER 2025,0
6,2025,WINTER,WINTER 2025,18
7,2026,FALL,FALL 2026,18
8,2026,SPRING,SPRING 2026,18
9,2026,SUMMER,SUMMER 2026,0


In [42]:
# Work on Generate Schedule

In [48]:
##async def generate_schedule(timedConnection, course_df, periods_df):

# initialize schedule    
schedule = []    
# keep track of max credits by term and year
#df = course_df
df.head()

,program_id,seq,name,course_type,type,credits,title,completed,term,year,session,locked,pre,pre_credits,substitutions,description,scheduled
0,10,1,LIBS 150,general,general,1,Introduction to Research,0,0,0,0,False,,,"COMP 111, LIBS 100, or LIBS 150",An introduction to the research process and me...,False
1,10,2,PACE 111T,general,general,3,Program and Career Exploration in Technology,0,0,0,0,False,,,"PACE 111B, PACE 111C, PACE 111M, PACE 111P, PA...",(Fulfills the general education requirement in...,False
2,10,3,WRTG 111,general,general,3,Academic Writing I,0,0,0,0,False,,,"WRTG 100A, WRTG 111, or WRTG 111X",(The first course in the two-course series WRT...,False
3,10,4,IFSM 201,general,general,3,Concepts and Applications of Information Techn...,0,0,0,0,False,,,"BMGT 301, CAPP 101, CAPP 300, CMST 300, IFSM 2...",(Access to a standard office productivity pack...,False
4,10,5,NUTR 100,general,general,3,Elements of Nutrition,0,0,0,0,False,,,NUTR 100 or NUTR 200,A study of the scientific and quantitative fou...,False


In [49]:
idx = 0
course = df.iloc[idx]
course

program_id                                                      10
seq                                                              1
name                                                      LIBS 150
course_type                                                general
type                                                       general
credits                                                          1
title                                     Introduction to Research
completed                                                        0
term                                                             0
year                                                             0
session                                                          0
locked                                                       False
pre                                                               
pre_credits                                                       
substitutions                      COMP 111, LIBS 100, or LIBS

In [50]:
assigned = False

In [55]:
# iterate over periods to schedule the course
period_idx = 0
periods_df.iloc[period_idx][['period']]
term_year = periods_df.iloc[period_idx][['period']]
term_year

period    SPRING 2024
Name: 0, dtype: object

In [57]:
(periods_df.at[period_idx, 'max_courses'] > 0) 

True

In [66]:
 max_credits_df[max_credits_df['period'] == 'SPRING 2024']['max_credits'].values[0]

18

In [ ]:
# Iterate over courses
for idx, course in df.iterrows():
    assigned = False
        
    # iterate over periods to schedule the course
    for period_idx, period in periods_df.iterrows():

            term_year = (period['term'], period['year'])

            if periods_df.at[period_idx, 'max_courses'] > 0 and max_credits_by_term_year.get(term_year, 0) >= course['credits']:
                # Add the course to the schedule
                schedule.append({
                    'seq': len(schedule) + 1,
                    'name': course['name'],
                    'term': period['term'],
                    'year': period['year'],
                    'session': period['session'],
                    'locked': False
                })
                    
                periods_df.at[period_idx, 'max_courses'] -= 1
                max_credits_by_term_year[term_year] -= course['credits']
                
                assigned = True
                break
        
        if not assigned:
            print(f"Unable to assign unlocked course '{course['name']}' to any period.")

    return pd.DataFrame(schedule)